# Create BRISE dataset for POTATO

In [1]:
from xpotato.dataset.dataset import Dataset
from xpotato.models.trainer import GraphTrainer

Load all data

In [142]:
import os
import json
import pandas as pd

GOLD_ATTRIBUTES = "gold_attributes"
GOLD = "labels_gold"
ANNOTATED_ATTRIBUTES = "annotated_attributes"
SEGMENTATION_ERROR = "segmentation_error"

def load_data(dir_path, only_gold=True):
    data = []
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename), "rt") as f:
            doc = json.load(f)
            if only_gold and not doc[GOLD]:
                continue
            for sen in doc["sens"].values():
                if sen[SEGMENTATION_ERROR]:
                    continue
                if doc[GOLD]:
                    labels = list(set(sen[GOLD_ATTRIBUTES].keys()))
                else:
                    labels = list(set(sen[ANNOTATED_ATTRIBUTES].keys()))
                data.append({
                    "sen_id": sen["id"],
                    "text": sen["text"],
                    "labels": labels,
                })
    df = pd.DataFrame.from_dict(data)
    print(df.shape)
    return df

## Load data

In [143]:
TRAIN_DIR = "../../../../data/train"
VAL_DIR = "../../../../data/valid"

train_data = load_data(TRAIN_DIR, only_gold=False)
val_data = load_data(VAL_DIR, only_gold=False)

(8917, 3)
(735, 3)


In [30]:
train_data[10:15]

,sen_id,text,labels
10,8025_7_0,Bestimmungen ohne Bezeichnung des Geltungsbere...,[Planzeichen]
11,8025_8_0,Für das gesamte Plangebiet wird bestimmt: Däch...,"[AbschlussDachMaxBezugGebaeude, PlangebietAllg..."
12,8025_9_0,Im Bauland/Wohngebiet darf die Bruttogeschoßfl...,"[WidmungUndZweckbestimmung, Flaechen]"
13,8025_10_0,Bestimmungen mit Bezeichnung des Geltungsberei...,[Planzeichen]
14,8025_11_0,Für die mit BB1 bezeichneten Grundflächen wird...,"[Planzeichen, UnterbrechungGeschlosseneBauweise]"


In [27]:
val_data[15:20]

,text,labels
sen_id,,
6969_9_0,"Die bebaubaren, von Bebauung freibleibenden Gr...",[AnordnungGaertnerischeAusgestaltung]
6969_9_1,Rangier- und Zufahrtsflächen sind nur im unbed...,[]
6969_10_0,Einfriedungen an seitlichen und hinteren Grund...,"[EinfriedungLage, AnordnungGaertnerischeAusges..."
6969_11_0,"Für alle Flächen, für die die gärtnerische Aus...","[VorkehrungBepflanzung, AnordnungGaertnerische..."
6969_12_0,Pro Bauplatz darf nur ein Nebengebäude bis zu ...,"[Flaechen, GebaeudeBautyp]"


## Generate labels

In [144]:
NOT = "NOT"

def get_labels(dfs):
    df_all = pd.concat(dfs)
    labels = pd.DataFrame(df_all["labels"].explode().unique(), columns=["label_name"])
    labels.fillna(NOT, inplace=True)
    labels['id'] = labels.index
    labels.set_index("label_name", inplace=True)
    print(labels.shape)
    return labels

In [8]:
unique_labels = get_labels([train_data, val_data])

(96, 1)


In [145]:
unique_labels.head()

,id
label_name,
NOT,0
OeffentlicheVerkehrsflaecheBreiteMin,1
GehsteigbreiteMin,2
AnFluchtlinie,3
VerkehrsflaecheID,4


In [15]:
unique_labels.to_dict()["id"]

{'NOT': 0,
 'OeffentlicheVerkehrsflaecheBreiteMin': 1,
 'GehsteigbreiteMin': 2,
 'AnFluchtlinie': 3,
 'VerkehrsflaecheID': 4,
 'VorkehrungBepflanzung': 5,
 'Planzeichen': 6,
 'AbschlussDachMaxBezugGebaeude': 7,
 'PlangebietAllgemein': 8,
 'GebaeudeHoeheArt': 9,
 'WidmungUndZweckbestimmung': 10,
 'Flaechen': 11,
 'UnterbrechungGeschlosseneBauweise': 12,
 'BegruenungDach': 13,
 'Dachart': 14,
 "DON'T ANNOTATE THIS SENTENCE": 15,
 'StrassenbreiteMin': 16,
 'WidmungInMehrerenEbenen': 17,
 'VorstehendeBauelementeAusladungMax': 18,
 'AnordnungGaertnerischeAusgestaltung': 19,
 'GebaeudeBautyp': 20,
 'AnlageZumEinstellenVorhanden': 21,
 'UnterirdischeBaulichkeiten': 22,
 'AnordnungGaertnerischeAusgestaltungProzentual': 23,
 'EinfriedungLage': 24,
 'EinfriedungHoeheGesamt': 25,
 'EinfriedungAusgestaltung': 26,
 'MaxAnzahlDachgeschosse': 27,
 'DachneigungMax': 28,
 'GebaeudeHoeheMax': 29,
 'DachflaecheMin': 30,
 'ErrichtungGebaeude': 31,
 'EinfriedungZulaessig': 32,
 'StrassenbreiteVonBis': 33,


## Generate dataset

In [31]:
TRAIN_GRAPH_UD = "train_ud.pickle"
VAL_GRAPH_UD = "val_ud.pickle"

TRAIN_GRAPH_4LANG = "train_4lang.pickle"
VAL_GRAPH_4LANG = "val_4lang.pickle"

TRAIN_GRAPH_GOLD_UD = "gold_train_ud.pickle"
VAL_GRAPH_GOLD_UD = "gold_val_ud.pickle"

TRAIN_GRAPH_GOLD_4LANG = "gold_train_4lang.pickle"
VAL_GRAPH_GOLD_4LANG = "gold_val_4lang.pickle"

def get_sentences(df):
    df_with_nots = df[["text"]]
    df_with_nots.loc[:, "dummy_label"] = NOT
    return df_with_nots.to_records(index=False)

import pickle

def get_graphs(dataset, graph_path, graph_format):
    if not os.path.exists(graph_path):
        dataset.set_graphs(dataset.parse_graphs(graph_format=graph_format))
    else:
        dataset.load_graphs(graph_path)
    df = dataset.to_dataframe()
    if not os.path.exists(graph_path):
        with open(graph_path, "wb") as f:
            pickle.dump(df.graph, f)
    return df

def get_dataset(data, vocab, graph_path, save_path, graph_format):
    print(f"Start processing data for {save_path} with {graph_format} graphs...")
    sentences = get_sentences(data)
    dataset = Dataset(sentences, label_vocab=vocab, lang="de")
    df = get_graphs(dataset, graph_path, graph_format)
    df.loc[:, "sen_id"] = data["sen_id"]
    df.loc[:, "labels"] = data["labels"]
    df.to_pickle(save_path)
    print(f"...finished processing data for {save_path} with {graph_format} graphs")
    return df

In [32]:
from tqdm import tqdm
from functools import partial

tqdm = partial(tqdm, position=0, leave=True)

vocab = unique_labels.to_dict()["id"]

# Train
## UD
TRAIN_UD = "train_ud"
train_complete_ud = get_dataset(train_data, vocab, TRAIN_GRAPH_UD, TRAIN_UD, "ud")
## UD
TRAIN_4LANG = "train_4lang"
train_complete_4lang = get_dataset(train_data, vocab, TRAIN_GRAPH_4LANG, TRAIN_4LANG, "fourlang")

# Valid
## UD
VAL_UD = "val_ud"
val_complete_ud = get_dataset(val_data, vocab, VAL_GRAPH_UD, VAL_UD, "ud")
## 4lang
VAL_4LANG = "val_4lang"
val_complete_4lang = get_dataset(val_data, vocab, VAL_GRAPH_4LANG, VAL_4LANG, "fourlang")

Start processing data for train_ud with ud graphs...


/home/eszter/.pyenv/versions/brise/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


...finished processing data for train_ud with ud graphs
Start processing data for train_4lang with fourlang graphs...
...finished processing data for train_4lang with fourlang graphs
Start processing data for val_ud with ud graphs...
...finished processing data for val_ud with ud graphs
Start processing data for val_4lang with fourlang graphs...
...finished processing data for val_4lang with fourlang graphs


In [33]:
train_complete_ud

,text,label,label_id,graph,sen_id,labels
0,MAGISTRAT DER STADT WIEN MA 21 Stadtteilplanun...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",8025_0_0,[]
1,Der Gemeinderat hat in seiner Sitzung am 29. A...,NOT,0,"(1, 2, 3, 0, 4, 6, 5, 7, 9, 8, 10, 11, 12, 14,...",8025_0_1,[]
2,"Bezirk, Kat. Großjedlersdorf I werden unter An...",NOT,0,"(1, 0, 2, 3, 4, 25, 5, 6, 7, 8, 9, 10, 11, 12,...",8025_1_0,[]
3,Die bisherigen Flächenwidmungspläne und Bebauu...,NOT,0,"(1, 3, 2, 6, 4, 5, 0, 7, 14, 8, 9, 10, 13, 11,...",8025_2_0,[]
4,Gemäß § 4 und § 5 der BO für Wien sowie § 48 d...,NOT,0,"(1, 2, 20, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...",8025_3_0,[]
...,...,...,...,...,...,...
8941,Die Dächer der zur Errichtung gelangenden Gebä...,NOT,0,"(1, 2, 18, 3, 8, 4, 6, 5, 7, 9, 10, 12, 11, 13...",7737_12_1,"[BegruenungDach, Dachart]"
8942,Dachterrassen sind bis zu einem Höchstausmaß v...,NOT,0,"(1, 13, 2, 3, 6, 4, 5, 7, 9, 8, 10, 12, 11, 0,...",7737_12_2,[]
8943,Technische bzw. der Belichtung dienende Aufbau...,NOT,0,"(1, 7, 2, 3, 5, 4, 6, 13, 8, 9, 12, 10, 11, 0,...",7737_12_3,[AufbautenZulaessig]
8944,Die mit G BB2 bezeichnete Fläche ist mit Ausna...,NOT,0,"(1, 6, 2, 3, 5, 4, 16, 7, 8, 9, 10, 14, 11, 12...",7737_13_0,"[Planzeichen, AnordnungGaertnerischeAusgestalt..."


In [34]:
train_complete_4lang

,text,label,label_id,graph,sen_id,labels
0,MAGISTRAT DER STADT WIEN MA 21 Stadtteilplanun...,NOT,0,"(0, 1, 2, 3, 4)",8025_0_0,[]
1,Der Gemeinderat hat in seiner Sitzung am 29. A...,NOT,0,"(5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",8025_0_1,[]
2,"Bezirk, Kat. Großjedlersdorf I werden unter An...",NOT,0,"(42, 43, 44, 15, 45, 46, 47, 48, 25, 49, 50, 5...",8025_1_0,[]
3,Die bisherigen Flächenwidmungspläne und Bebauu...,NOT,0,"(56, 57, 58, 59, 60, 61, 62, 63, 64, 7, 65, 66...",8025_2_0,[]
4,Gemäß § 4 und § 5 der BO für Wien sowie § 48 d...,NOT,0,"(42, 43, 33, 68, 7, 52, 54, 25, 49, 69, 70, 71...",8025_3_0,[]
...,...,...,...,...,...,...
8941,Die Dächer der zur Errichtung gelangenden Gebä...,NOT,0,"(397, 94, 145, 146, 279, 209, 206, 12, 147, 29...",7737_12_1,"[BegruenungDach, Dachart]"
8942,Dachterrassen sind bis zu einem Höchstausmaß v...,NOT,0,"(132, 699, 207, 2085, 209, 84, 1397, 697, 287)",7737_12_2,[]
8943,Technische bzw. der Belichtung dienende Aufbau...,NOT,0,"(132, 404, 405, 406, 109, 143, 15, 256, 271)",7737_12_3,[AufbautenZulaessig]
8944,Die mit G BB2 bezeichnete Fläche ist mit Ausna...,NOT,0,"(105, 94, 230, 227, 11, 31, 228, 148, 231, 137...",7737_13_0,"[Planzeichen, AnordnungGaertnerischeAusgestalt..."


In [36]:
val_complete_ud

,text,label,label_id,graph,sen_id,labels
0,Unverkäufliches Dienstexemplar!,NOT,0,"(1, 2, 0, 3)",6969_0_0,[DON'T ANNOTATE THIS SENTENCE]
1,M A G I S T R A T D E R S T A D T W I E N MA 2...,NOT,0,"(1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",6969_0_1,[DON'T ANNOTATE THIS SENTENCE]
2,Der Gemeinderat hat in seiner Sitzung am 14. D...,NOT,0,"(1, 2, 3, 0, 4, 6, 5, 7, 9, 8, 10, 12, 11, 13,...",6969_0_2,[DON'T ANNOTATE THIS SENTENCE]
3,"Bezirk, Kat. G. Breitensee sowie in Festsetzun...",NOT,0,"(1, 38, 2, 5, 3, 4, 6, 7, 9, 8, 10, 11, 12, 13...",6969_0_3,[DON'T ANNOTATE THIS SENTENCE]
4,Soweit ein gültiger Flächenwidmungsplan und Be...,NOT,0,"(1, 7, 2, 4, 3, 5, 6, 9, 8, 0, 10, 11, 13, 12,...",6969_1_0,[]
...,...,...,...,...,...,...
731,Der mit Eklw P BB5 bezeichnete Bereich ist ein...,NOT,0,"(1, 7, 2, 3, 6, 4, 5, 22, 8, 9, 10, 11, 13, 12...",7094k_5_0,"[Planzeichen, AnlageZumEinstellenVorhanden, Ge..."
732,"Auf den mit BB8 bezeichneten Flächen, dürfen K...",NOT,0,"(1, 6, 2, 3, 4, 5, 18, 7, 8, 9, 10, 11, 12, 13...",7094k_6_0,"[Planzeichen, GebaeudeBautyp]"
733,Im Übrigen haben die Planzeichen die Bedeutung...,NOT,0,"(1, 3, 2, 4, 0, 5, 6, 7, 8, 9, 10, 11, 12, 14,...",7094k_7_0,[]
734,Der Abteilungsleiter: Dipl.-Ing.,NOT,0,"(1, 2, 0, 3, 4, 5, 6)",7094k_7_1,[]


In [37]:
val_complete_4lang

,text,label,label_id,graph,sen_id,labels
0,Unverkäufliches Dienstexemplar!,NOT,0,"(0, 1)",6969_0_0,[DON'T ANNOTATE THIS SENTENCE]
1,M A G I S T R A T D E R S T A D T W I E N MA 2...,NOT,0,"(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",6969_0_1,[DON'T ANNOTATE THIS SENTENCE]
2,Der Gemeinderat hat in seiner Sitzung am 14. D...,NOT,0,"(22, 23, 2, 24, 25, 26, 27, 28, 29, 30, 31, 32...",6969_0_2,[DON'T ANNOTATE THIS SENTENCE]
3,"Bezirk, Kat. G. Breitensee sowie in Festsetzun...",NOT,0,"(73, 74, 63, 2, 54, 75, 76, 56, 77, 78, 79, 80...",6969_0_3,[DON'T ANNOTATE THIS SENTENCE]
4,Soweit ein gültiger Flächenwidmungsplan und Be...,NOT,0,"(89, 90, 91, 92, 93, 2, 94, 95, 96, 97)",6969_1_0,[]
...,...,...,...,...,...,...
731,Der mit Eklw P BB5 bezeichnete Bereich ist ein...,NOT,0,"(401, 383, 165, 132, 389, 468, 844, 156, 27, 6...",7094k_5_0,"[Planzeichen, AnlageZumEinstellenVorhanden, Ge..."
732,"Auf den mit BB8 bezeichneten Flächen, dürfen K...",NOT,0,"(219, 1055, 66, 1131, 132, 913, 198, 2, 1132, ...",7094k_6_0,"[Planzeichen, GebaeudeBautyp]"
733,Im Übrigen haben die Planzeichen die Bedeutung...,NOT,0,"(64, 102, 299, 92, 170, 300, 301, 132, 279, 32...",7094k_7_0,[]
734,Der Abteilungsleiter: Dipl.-Ing.,NOT,0,"(266, 267, 271, 272)",7094k_7_1,[]


In [100]:
def set_label(row, attribute):
    if attribute in row["labels"]:
        return attribute
    else:
        return NOT

def set_label_id(row, attribute, labels):
    if row["label"] == attribute:
        return labels.loc[attribute]["id"]
    else:
        return labels.loc[NOT]["id"]

def set_one_vs_all(df, labels, attribute):
    new = df.copy()
    new["label"] = new.apply(lambda x: set_label(x, attribute), axis=1)
    new["label_id"] = new.apply(lambda x: set_label_id(x, attribute, labels), axis=1)
    return new

## WidmungUndZweckbestimmung

In [96]:
WUZ = "WidmungUndZweckbestimmung"

### 4lang

In [105]:
path = "train_wuz_4lang"

df_train = set_one_vs_all(train_complete_4lang, unique_labels, WUZ)
df_train.to_pickle(path)

In [107]:
path = "val_wuz_4lang"

df_val = set_one_vs_all(val_complete_4lang, unique_labels, WUZ)
df_val.to_pickle(path)

In [109]:
df_train.loc[df_train["label"] != NOT].head()

,text,label,label_id,graph,sen_id,labels
12,Im Bauland/Wohngebiet darf die Bruttogeschoßfl...,WidmungUndZweckbestimmung,10,"(121, 97, 122, 123, 124, 125, 126, 59, 7, 127,...",8025_9_0,"[WidmungUndZweckbestimmung, Flaechen]"
28,"Nicht bebaute, aber bebaubare Flächen im Baula...",WidmungUndZweckbestimmung,10,"(105, 94, 132, 206, 84, 7, 234, 235, 236, 237,...",7752_6_0,"[WidmungUndZweckbestimmung, AnordnungGaertneri..."
46,Auf der mit BB9 bezeichneten Fläche dürfen nur...,WidmungUndZweckbestimmung,10,"(239, 146, 305, 31, 281, 258, 84, 97, 306, 307...",7752_20_0,"[WidmungUndZweckbestimmung, Planzeichen, Gebae..."
47,Auf der mit BB10 bezeichneten Fläche dürfen Ge...,WidmungUndZweckbestimmung,10,"(239, 309, 218, 132, 219, 97, 283, 7, 281, 310...",7752_21_0,"[WidmungUndZweckbestimmung, Flaechen, Planzeic..."
79,Innerhalb der mit BB1 bezeichneten und als Bau...,WidmungUndZweckbestimmung,10,"(218, 219, 413, 225, 403, 414, 325, 221, 132, ...",7498_13_0,"[WidmungUndZweckbestimmung, Flaechen, Planzeic..."


In [112]:
df_val.loc[df_val["label"] != NOT].head()

,text,label,label_id,graph,sen_id,labels
28,Der im Grünland - Erholungsgebiet/Kleingarteng...,WidmungUndZweckbestimmung,10,"(252, 156, 27, 56, 253, 254, 61, 250, 115, 30,...",6969_17_0,"[WidmungUndZweckbestimmung, Planzeichen]"
29,Die bebaute Fläche der als Gemeinschaftsanlage...,WidmungUndZweckbestimmung,10,"(258, 198, 122, 259, 260, 159, 217, 261, 184, ...",6969_17_1,"[WidmungUndZweckbestimmung, Flaechen]"
60,Auf der mit BB 2 bezeichneten und als Verkehrs...,WidmungUndZweckbestimmung,10,"(2, 341, 118, 342, 343, 173, 61, 129, 344, 132...",7086_7_0,"[WidmungUndZweckbestimmung, Planzeichen, Durch..."
81,Innerhalb der als Bauland gewidmeten und mit G...,WidmungUndZweckbestimmung,10,"(383, 165, 132, 389, 216, 390, 217, 2, 27, 61,...",6833_12_0,"[WidmungUndZweckbestimmung, Planzeichen, VonBe..."
83,In den mit P/BB3 bezeichneten Bereichen im Bau...,WidmungUndZweckbestimmung,10,"(159, 394, 161, 395, 132, 396, 30, 347, 156, 2...",6833_14_0,"[StellplatzImNiveauZulaessig, AusnahmeGaertner..."


#### Train

In [113]:
trainer = GraphTrainer(df_train)
features = trainer.prepare_and_train()

Initializing trainer object...
Featurizing graphs by generating subgraphs up to 2...


8946it [00:53, 167.95it/s]


Getting feature graphs...
Selecting the best features...
Generating training data...
Training...
Getting features...


/home/eszter/.pyenv/versions/brise/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [125]:
features[WUZ] = features[WUZ][:20]

In [126]:
features

defaultdict(list,
            {'WidmungUndZweckbestimmung': [(['(u_488 / vorbehalt)'],
               [],
               'WidmungUndZweckbestimmung'),
              (['(u_309 / Nutzung)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_471 / Baugebiet)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_133 / BaulandSLASHWohngebiet)'],
               [],
               'WidmungUndZweckbestimmung'),
              (['(u_592 / vorbehalten)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_12 / als)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_687 / dienen)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_457 / Vorsorge)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_1125 / ueberwiegend)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_486 / Z)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_244 / Bauland)'], [], 'WidmungUndZweckbestimmung'),
              (['(u_1014 / Befestigung)'], [], 'WidmungUndZweckbestimmung'),
      

In [127]:
with open("features_wuz_4lang.json", "w") as f:
    json.dump(features, f)

Call with streamlit: 

```bash
streamlit run frontend/app.py -- -t /home/eszter/research/brise-plandok/brise_plandok/full_attribute_extraction/attribute/potato/train_wuz_4lang -v /home/eszter/research/brise-plandok/brise_plandok/full_attribute_extraction/attribute/potato/val_wuz_4lang -g fourlang -sr /home/eszter/research/brise-plandok/brise_plandok/full_attribute_extraction/attribute/potato/features_wuz_4lang.json
```

#### Predict

In [115]:
from xpotato.graph_extractor.extract import FeatureEvaluator

def get_pred_df(df, features):
    feature_values = []
    for k in features:
        for f in features[k]:
            feature_values.append(f)
    evaluator = FeatureEvaluator()
    pred_df = evaluator.match_features(df, feature_values)
    if "label" in df:
        pred_df["label"] = df.label
    return pred_df

In [116]:
pred_df = get_pred_df(df_val, features)

736it [00:13, 52.68it/s]


In [117]:
pred_df.head()

,Sentence,Predicted label,Matched rule,label
0,Unverkäufliches Dienstexemplar!,,,NOT
1,M A G I S T R A T D E R S T A D T W I E N MA 2...,WidmungUndZweckbestimmung,"([(u_7 / COORD)], [], WidmungUndZweckbestimmung)",NOT
2,Der Gemeinderat hat in seiner Sitzung am 14. D...,WidmungUndZweckbestimmung,"([(u_12 / als)], [], WidmungUndZweckbestimmung)",NOT
3,"Bezirk, Kat. G. Breitensee sowie in Festsetzun...",WidmungUndZweckbestimmung,"([(u_25 / fuer)], [], WidmungUndZweckbestimmung)",NOT
4,Soweit ein gültiger Flächenwidmungsplan und Be...,WidmungUndZweckbestimmung,"([(u_113 / er)], [], WidmungUndZweckbestimmung)",NOT


#### Evaluate

In [118]:
evaluator = FeatureEvaluator()

In [119]:
eval_df = evaluator.evaluate_feature(WUZ, features[WUZ], df_val)

/home/eszter/.pyenv/versions/brise/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/eszter/.pyenv/versions/brise/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/eszter/.pyenv/versions/brise/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [22]:
eval_df[0][["Feature", "Precision", "Recall", "Fscore"]][:15]

,Feature,Precision,Recall,Fscore
0,[(u_38 / bezeichneten)],0.666667,0.333333,0.444444
1,[(u_293 / Flaechen)],0.588235,0.416667,0.487805
2,[(u_248 / Flaeche)],0.666667,0.250000,0.363636
3,[(u_80 / PERIOD)],0.338235,0.958333,0.500000
4,[(u_17 / COLON)],0.076923,0.083333,0.080000
5,[(u_394 / darueber)],0.000000,0.000000,0.000000
6,[(u_310 / auszubilden)],0.750000,0.125000,0.214286
7,[(u_391 / Verkehrsflaeche)],1.000000,0.083333,0.153846
8,[(u_26 / mit)],0.535714,0.625000,0.576923
9,[(u_142 / BB)],0.250000,0.041667,0.071429


In [141]:
UD = "ud"
FOURLANG = "4lang"

CONSTANTS = { 
    UD : {
        "train": train_complete_ud,
        "val": val_complete_ud,
    },
    FOURLANG : {
        "train": train_complete_4lang,
        "val": val_complete_4lang,
    },
}

GRAPH_MAP = {
    UD: UD,
    FOURLANG: "fourlang",
}

def get_dataset_path(dataset, attribute, graph):
    return f"{dataset}_{attribute}_{graph}"

def get_feature_path(attribute, graph):
    return f"features_{attribute}_{graph}.json"

def print_streamlit_command(attribute, graph):
    root = "/home/eszter/research/brise-plandok/brise_plandok/full_attribute_extraction/attribute/potato"
    train_data = f"{root}/{get_dataset_path('train', attribute, graph)}"
    valid_data = f"{root}/{get_dataset_path('val', attribute, graph)}"
    features = f"{root}/{get_feature_path(attribute, graph)}"
    print(f"streamlit run frontend/app.py -- -t {train_data} -v {valid_data} -g {GRAPH_MAP[graph]} -sr {features}")
    
def create_label_spec_dataset(dataset, attribute, graph):
    path = et_dataset_path(dataset, attribute, graph)
    df = set_one_vs_all(CONSTANTS[graph][dataset], unique_labels, attribute)
    df.to_pickle(path)
    return df

def train_and_save(attribute, graph):
    df_train = create_label_spec_dataset("train", attribute, graph)
    df_val = create_label_spec_dataset("val", attribute, graph)
    trainer = GraphTrainer(df_train)
    features = trainer.prepare_and_train()
    with open(get_feature_path(attribute, graph), "w") as f:
        json.dump(features, f)
    print_streamlit_command(attribute, graph)

## Planzeichen

In [139]:
PLZ = "Planzeichen"

In [135]:
train_and_save(PLZ, FOURLANG)

Initializing trainer object...
Featurizing graphs by generating subgraphs up to 2...


8946it [00:52, 169.16it/s]


Getting feature graphs...
Selecting the best features...
Generating training data...
Training...
Getting features...


In [140]:
print_streamlit_command(PLZ, FOURLANG)

streamlit run frontend/app.py -- -t /home/eszter/research/brise-plandok/brise_plandok/full_attribute_extraction/attribute/potato/train_Planzeichen_4lang -v /home/eszter/research/brise-plandok/brise_plandok/full_attribute_extraction/attribute/potato/val_Planzeichen_4lang -g fourlang -sr /home/eszter/research/brise-plandok/brise_plandok/full_attribute_extraction/attribute/potato/features_Planzeichen_4lang.json
